Seems like online source coding is a specific case of online portfolio optimization see sources for earlier work.

 # Online Fibonacci coding idea

imagine we have stream of char ABCDEF and we want to use an encoding to minimize the length of the stream in bits, for the IID case this can be solved with high probability by assigning low length codings to frequent char this is normal Fibonacci coding, but if the frequencies aren't know or worse the stream is adversarial we frame this problem as an online linear optimization problem.     

# fractional Fibonacci coding

Like we needed fractional caching to formulate caching as an OLO, we also do some convexification trick for Fibonacci coding, in normal encoding you assign each character a code instead we reduce this code to only the code length as this doesn't change our performance metric (given characters with the same code length you can find a scheme to decide which character gets assign to which code), now in fractional encoding you assign a fraction of a character to each encoding length in general we can represent all the fractional encodings of the characters in following tensor $y^{cl}$ where $c$ indexes the character and $l$ indexes the code length. To be a feasible fractional encoding $y$ has to satisfy following conditions:
- probability distribution over code lengths for each character
$$
\forall c: \sum_{l} y^{cl} = 1, \forall  c,l :y^{cl} \ge 0.
$$ 
- restriction on the amount of code lengths for each size 
$$
\forall l: \sum_{c} y^{cl} \le F(l-1)
$$ 
with $F(n)$ the $n$the Fibonacci number.

for the fractional encoding the loss for an observed character is the average code length of its representations:
$$
l(c) = \sum_{l}y^{cl}l .
$$ 
which is linear in the tensor $y$.

## doing compression with fractional coding

For a batch of characters or for each character the encoder and decoder does the exact same rounding scheme (this is possible by using the same seed), a sample of a integral encoding i.e. a $R[y]$ where $\forall c,l: R[y]^{cl} \in \{0,1\}$ and $E[R[y]] = y$ then encoding and decoding is as simple as the integral case. 

TODO: study independent and coupled rounding similar to fractional caching problem

## Optimality of universal Fractional coding

Integral Fibonacci code is only optimal for its implied character distribution the one with 
$\forall l \in \mathbb{N}:$ there are $F(l-1)$ characters with probability $2^{-l}$

Fractional Fibonnaci coding is optimal for more then $1$ distribution.

We expect that this optimality isn't extended far, distributions with characters with probability above $0.25$
can't be optimal in Fibonacci coding.

## sources Fibonacci encoding

https://en.wikipedia.org/wiki/Fibonacci_coding
https://en.wikipedia.org/wiki/Universal_code_(data_compression)
https://en.wikipedia.org/wiki/Shannon%27s_source_coding_theorem

# Online Arithmetic coding

Static arithmetic coding requires the frequencies of the encoded characters. Let's look first at the static case in hindsight an adversarial choses a sequence of characters $c_{t}$ and we want to minimize the code length of arithmetic coding, a good bound for this is entropy + cte i.e. we can directly minimize entropy, the feasible set is a distribution i.e. where $y^{c}$ represent the frequency for character $c$:

$$
Y = \left\{y \in [0,1]^{N} \middle| \sum_{c} y^{c} = 1 \right\}.
$$ 

the static problem in hindsight is following convex program:

$$
\min_{y \in Y}\left(\frac{1}{T} \sum_{t=1}^{T} -\log\left(y^{c_{t}}\right)\right).
$$ 

OCO doesn't directly apply because the gradients aren't bounded, to fix this we limit $Y$:
(maybe because our loss in exponentially concave we can apply ONS, no idea about fast ONS)

$$
Y = \left\{y \in [\varepsilon,0.5]^{N} \middle| \sum_{c} y^{c} = 1 \right\},
$$ 

$\varepsilon =1/512 $ for example ensures bounded gradients and  that  $y$ can be represented as a UInt8 value. This would limit our alphabet size to $256$ distinct characters (it isn't $512$ as this only has $1$ element in $Y$) . For practical extension beyond this limit, you could implement a "combination character" scheme, similar to the Shift key on a keyboard. When this character is encountered, the subsequent few characters are interpreted as an encoding for a character outside the standard $256$-character alphabet. To approximately decide how to do this higher level encoding you can use an online caching algorithm. 

We think being adaptive to very infrequent characters is a fundamentally flawed as you lack the information to estimate/learn about them per definition. We should prove some kind of theorem like the entropy is decided by the characters you can learn and estimate and the rest are at worst will add uniform entropy without estimating them. 


A different way of getting bounded gradients is by providing a maximum encoding length of all characters by 
using a literal symbol and sending the bit representation.  

## quantized online arithmetic coding

It is possible to apply the same quantized algorithm for caching to arithmetic coding !!!

But we haven't figured out how to implement arithmetic coding with a lazy subtraction so we
can maintain $O(1)$ cost amortized per step, $O(log N)$ where $N$ is the amount of distinct characters 
would also be good compared classic adaptive algorithms.

## OGD for calculating dynamic entropy

The goal for the only calculating the dynamic entropy is to understand the performance and sub goal for a full implementation. 


## sources online source coding

Soft-Bayes: Prod for Mixtures of Experts with Log-Loss: https://arxiv.org/pdf/1901.02230
